In [ ]:
# Define qual implementação de MapReduce usar: 'java' ou 'python'
MAPREDUCE_LANG = 'java' # Altere para 'python' para usar a implementação em Python

print(f"Usando implementação de MapReduce para Predição de Vacinação: {MAPREDUCE_LANG.upper()}")

# Lab 4: Pipeline Interativo de Predição do Ciclo de Vacinação

Este notebook executa o pipeline de predição do ciclo de vacinação para cães e gatos. Ele se conecta ao banco de dados PostgreSQL, calcula as próximas doses e identifica vacinas vencidas.

## 1. Compilando a Aplicação MapReduce

In [ ]:
%%bash
if [ "$MAPREDUCE_LANG" == "java" ]; then
    cd labs/mapreduce-jobs/vaccination-prediction
    mvn package -q
    echo "JAR compilado com sucesso em: target/vaccination-prediction-1.0-SNAPSHOT.jar"
fi

## 2. Conexão ao Banco de Dados e Configuração

In [ ]:
import psycopg2
import os
from datetime import datetime, timedelta

# As credenciais e o host são baseados no arquivo docker-compose.txt
DB_HOST = "localhost" # Nome do serviço no Docker Compose
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

def get_db_connection():
    return psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)

print("Configuração do banco de dados carregada.")

## 3. Ingestão de Dados com Sqoop para MapReduce

In [ ]:
%%bash
INPUT_DIR=/petshop/input_vaccination_records

# Remove o diretório de entrada se ele já existir
hdfs dfs -test -d $INPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS existente: $INPUT_DIR"
    hdfs dfs -rm -r $INPUT_DIR
fi

echo "Iniciando importação com Sqoop..."
sqoop import \
    --connect jdbc:postgresql://localhost:5432/postgres \
    --username postgres \
    --password postgres \
    --query "SELECT vr.vaccination_record_id, p.pet_id, p.name, p.species, vr.application_date, vrf.vaccine_name, vrf.booster_interval_months, vrf.mandatory FROM vaccination_record vr JOIN pet p ON vr.pet_id = p.pet_id JOIN vaccine_reference vrf ON vr.vaccine_reference_id = vrf.vaccine_reference_id WHERE \$CONDITIONS" \
    --target-dir $INPUT_DIR \
    --m 1 \
    --split-by vr.vaccination_record_id

echo "\nImportação concluída. Verificando os dados no HDFS:"
hdfs dfs -ls $INPUT_DIR
hdfs dfs -cat $INPUT_DIR/part-m-00000 | head -n 5

## 4. Execução do Job MapReduce para Predição de Vacinação

In [ ]:
%%bash
INPUT_DIR=/petshop/input_vaccination_records
OUTPUT_DIR=/petshop/output_vaccination_predictions

# Remove o diretório de saída se ele já existir
hdfs dfs -test -d $OUTPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS de saída existente: $OUTPUT_DIR"
    hdfs dfs -rm -r $OUTPUT_DIR
fi

if [ "$MAPREDUCE_LANG" == "java" ]; then
    JAR_PATH=labs/mapreduce-jobs/vaccination-prediction/target/vaccination-prediction-1.0-SNAPSHOT.jar
    echo "Executando o job MapReduce (Java)...
    hadoop jar $JAR_PATH $INPUT_DIR $OUTPUT_DIR
elif [ "$MAPREDUCE_LANG" == "python" ]; then
    MAPPER_PATH=labs/mapreduce-jobs/vaccination-prediction-python/mapper.py
    REDUCER_PATH=labs/mapreduce-jobs/vaccination-prediction-python/reducer.py
    echo "Executando o job MapReduce (Python)...
    hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
        -file $MAPPER_PATH -mapper $MAPPER_PATH \
        -file $REDUCER_PATH -reducer $REDUCER_PATH \
        -input $INPUT_DIR \
        -output $OUTPUT_DIR
fi

echo "\nJob de Predição de Vacinação concluído!"

## 5. Verificação do Resultado do MapReduce

In [ ]:
%%bash
OUTPUT_DIR=/petshop/output_vaccination_predictions
echo "Conteúdo do diretório de saída:"
hdfs dfs -ls $OUTPUT_DIR

echo "\nResultado do processamento:"
hdfs dfs -cat $OUTPUT_DIR/part-r-00000

## 6. Carregando e Consultando no Redis (Exemplo)

In [ ]:
import redis
redis = redis.Redis(host = 'localhost', port=6379)
redis.ping()

In [ ]:
%%bash
OUTPUT_FILE=/petshop/output_vaccination_predictions/part-r-00000
REDIS_HOST=localhost

# Verifica se o arquivo de resultado existe
hdfs dfs -test -e $OUTPUT_FILE
if [ $? -ne 0 ]; then
    echo "Erro: Arquivo de resultado não encontrado em $OUTPUT_FILE"
    exit 1
fi

echo "Limpando recomendações antigas no Redis..."
redis-cli -h $REDIS_HOST KEYS "recomendacao:vaccination:pet:*" | xargs -r redis-cli -h $REDIS_HOST DEL

echo "Carregando novas recomendações do HDFS para o Redis..."
# Lê o arquivo do HDFS e processa linha por linha
hdfs dfs -cat $OUTPUT_FILE | while IFS=$'\t' read -r pet_id values; do
    # Extrai os valores (vaccine_name, next_dose_date, is_overdue, mandatory)
    vaccine_name=$(echo $values | cut -d',' -f1)
    next_dose_date=$(echo $values | cut -d',' -f2)
    is_overdue=$(echo $values | cut -d',' -f3)
    mandatory=$(echo $values | cut -d',' -f4)
    
    # Monta e executa o comando HSET para o Redis
    echo "Inserindo predição de vacinação para o pet_id: $pet_id, vacina: $vaccine_name"
    redis-cli -h $REDIS_HOST HSET "recomendacao:vaccination:pet:$pet_id" "$vaccine_name" \
        next_dose_date "$next_dose_date" \
        is_overdue "$is_overdue" \
        mandatory "$mandatory"
done

echo "\nCarga de dados concluída."

echo "Totos os registros inseridos:"
redis-cli -h $REDIS_HOST KEYS "recomendacao:vaccination:pet:*"
redis-cli -h $REDIS_HOST HGETALL "recomendacao:vaccination:pet:1" # Example for pet_id 1

## 7. Identificar Próximas Vacinas Recomendadas (Baseado na Idade e Espécie)

In [ ]:
def get_recommended_vaccines_for_pet(pet_id):
    conn = None
    cur = None
    try:
        conn = get_db_connection()
        cur = conn.cursor()

        # Get pet details and age
        cur.execute("SELECT species, EXTRACT(EPOCH FROM (AGE(NOW(), birth_date))) / 2592000 AS age_months FROM pet WHERE pet_id = %s;", (pet_id,))
        pet_info = cur.fetchone()

        if not pet_info:
            print(f"Pet com ID {pet_id} não encontrado.")
            return []

        species, age_months = pet_info

        # Get vaccines already applied to the pet
        cur.execute("SELECT vaccine_reference_id FROM vaccination_record WHERE pet_id = %s;", (pet_id,))
        applied_vaccine_ids = {row[0] for row in cur.fetchall()}

        # Get all vaccine references relevant to the pet's species and age
        query = """
        SELECT
            vaccine_name,
            description,
            mandatory,
            first_dose_age_months,
            booster_interval_months
        FROM vaccine_reference
        WHERE (target_species = %s OR target_species = 'Ambos')
          AND first_dose_age_months <= %s
          AND vaccine_reference_id NOT IN (SELECT vaccine_reference_id FROM vaccination_record WHERE pet_id = %s);
        """
        cur.execute(query, (species, age_months, pet_id))
        recommended_vaccines = cur.fetchall()

        # Filter out vaccines that have already been applied or are not yet due
        filtered_recommendations = []
        for rec in recommended_vaccines:
            (vaccine_name, description, mandatory, first_dose_age_months, booster_interval_months) = rec
            # For simplicity, this example assumes a single dose or the first dose is being recommended
            # More complex logic would be needed to track booster schedules for *unapplied* vaccines
            filtered_recommendations.append({
                'vaccine_name': vaccine_name,
                'description': description,
                'mandatory': mandatory,
                'first_dose_age_months': first_dose_age_months,
                'booster_interval_months': booster_interval_months
            })
        return filtered_recommendations

    except psycopg2.OperationalError as e:
        print(f"Erro de conexão: {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

# Exemplo de uso:
# Supondo que 'Bidu' (pet_id=1) tenha uma data de nascimento para calcular a idade
# Para este exemplo, vamos adicionar uma coluna 'birth_date' à tabela 'pet' para simular a idade.
# Você precisaria atualizar o DDL e DML em lab1-postgresql-setup.ipynb para incluir 'birth_date'.

# Para fins de demonstração, vamos simular um pet_id e uma idade.
# Para um uso real, a tabela 'pet' precisaria de uma coluna 'birth_date'.

print("\nRecomendações de Vacinas para Pet (Exemplo - requer 'birth_date' na tabela pet):")
# Para testar esta função, você precisaria adicionar uma coluna `birth_date` à tabela `pet`
# e popular com dados. Por exemplo:
# ALTER TABLE pet ADD COLUMN birth_date DATE;
# UPDATE pet SET birth_date = '2025-01-01' WHERE pet_id = 1;
# UPDATE pet SET birth_date = '2025-02-15' WHERE pet_id = 2;
# UPDATE pet SET birth_date = '2024-10-20' WHERE pet_id = 3;

# Exemplo de chamada (assumindo que pet_id 1 existe e tem birth_date):
# recommended = get_recommended_vaccines_for_pet(1)
# if recommended:
#     for rec in recommended:
#         print(rec)
# else:
#     print("Nenhuma vacina recomendada ou pet não encontrado.")